In [2]:

import os, glob
import pickle
import pymorphy2
import nltk
import numpy as np
import re
from typing import List
import collections
import datetime
from cytoolz import pipe
from tqdm import tqdm

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stopwords = nltk.corpus.stopwords.words(
    'russian') + nltk.corpus.stopwords.words('english')
stopwords += [
    'отличный', 'метр', 'наш', 'клиент', 'банка', 'проект', 'литр',
    'желательный', 'др', 'самый', 'мочь', 'хороший', 'год', 'чел', 'обязательный'
]

cache = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def _get_POS(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].tag.POS


def normal_form(word):
    if word not in cache:
        cache[word] = morph.parse(word)[0]
    return cache[word].normal_form


def is_word_pos_in(word: str, pos: List[str] = None) -> bool:
    if not pos:
        pos = ['NOUN', "ADJF", 'INFN', 'VERB', 'ADJS']

    return _get_POS(word) in pos


def get_words(text):
    return re.findall(r'\w+', text)


def nonempty(x):
    if isinstance(x, Sequence):
        return filter(lambda x: len(x) > 0 and x != ' ', x)
    return x

helper = {}


def remove_numbers(text):
    return re.sub(r'\d+', '', text)


def normalize_skill(skill: str):
    parsed = tuple(
        pipe(
            skill,
            lambda x: x.lower(),
            remove_numbers,
            get_words,
        ))
    
    clear_skill = []
    dirty_skill = []
    
    # Последнее стоп слово для dirty_skill
    last_stopword = None
    
    # Для каждого слова в скилле
    for i in parsed:
        # Нормализуем слово
        word = normal_form(i)
        
        # Если стоп слово - запомним его
        if word in nltk.corpus.stopwords.words('russian'):
            last_stopword = word
            
            if word == "без":
                clear_skill.append(word)
        
        # Проверим на часть речи, длинну и стоплова
        elif is_word_pos_in(word) and len(word) > 3 and word not in stopwords:
            
            # Если до этого было стоп слово, добавим его в dirty
            if last_stopword and len(dirty_skill) > 0:
                dirty_skill.append(last_stopword)
                last_stopword = None
            
            # Добавим в чистый скилл слово
            clear_skill.append(word)
            
            if is_word_pos_in(word, ['NOUN', 'ADJF']):
                dirty_skill.append(i)
    #f len(clear_skill) > 1 and len(clear_skill) < 8:
    return clear_skill, dirty_skill

In [ ]:
tmp_vector = np.array([0] * 300, dtype=np.float32)
def _word2vec(word):
    for i in ["_NOUN", "_ADJ", "_VERB"]:
        tmp = "{}{}".format(word, i)
        if tmp in model:
            return model[tmp]
        else:
            return tmp_vector
skill_to_vec = lambda x: np.mean(list(map(_word2vec, x)))

In [ ]:
def add_clear_vac_name(vac):
    vac = vac.copy()
    vac['profession_tree_name_vec'] = skill_to_vec(normalize_skill(vac['profession_tree_name'])[1])
    return vac

In [ ]:
_word2vec(normalize_skill(data[2500]['profession_tree_name'])[0][0])

In [ ]:
def find_vac_number(prof):
    norm_prof=normalize_skill(prof)
    for i in  range(len(data)):
        if(data[i]['profession_normalized'] == norm_prof or data[i]['profession_tree_name_normalized'] == norm_prof):
            return i
    return -1

In [22]:
resume = pickle.load(open("resume.pck", "rb"))
vacancy = pickle.load(open("vacancy.pck", "rb"))

In [43]:
def get_jobs(resume_id):
    ans = {}
    for job in resume[resume_id]['work_history']:
        ans[int(job['monthbeg'])/12 + int(job['yearbeg'])] = job
    return dict(collections.OrderedDict(sorted(ans.items())))

In [51]:
#get_jobs(128945)
resume[1]

{'additional_education': [{'institute': 'Японский образовательный центр менеджмента при “МИРБС”',
   'monthend': '11',
   'name': 'маркетинг',
   'town': 'Москва',
   'yearend': '1999'},
  {'institute': 'учебный центр  "Феликс "',
   'monthend': '2',
   'name': 'общий аудит',
   'town': 'Москва',
   'yearend': '1998'}],
 'age': 62,
 'base_education': [{'education_form': 'Не имеет значения',
   'education_type': 'Высшее',
   'faculty': 'экономический',
   'institute': 'Аспирантура Московского финансового института',
   'monthend': '7',
   'profession': 'экономист',
   'town': 'Москва',
   'yearend': '1986'},
  {'education_form': 'Не имеет значения',
   'education_type': 'Высшее',
   'faculty': 'экономический',
   'institute': 'Московский государственный университет им. М.В. Ломоносова',
   'monthend': '8',
   'profession': 'экономист',
   'town': 'Москва',
   'yearend': '1980'}],
 'best': 'Компьютерные навыки:\n MS Windows 98/2000/ME, MS Word, MS Excel, MS Outlook, Электронная почта, На

In [52]:
for i in tqdm(range(len(resume))):
    try:
        resume[i]['work_history'] = get_jobs(i)
    except:
        print(i)
        break


100%|██████████| 128945/128945 [00:00<00:00, 169959.70it/s]

In [30]:
def f():
    for res in tqdm(resume):
        for work in res['work_history']:
            #try:
                if(work['yearend'] == None or work['yearend'] == '0'):
                    work['yearend'] = datetime.datetime.now().strftime("%Y")
                    work['monthend'] = datetime.datetime.now().strftime("%m")
                if(work['yearbeg'] == None):
                    work['yearbeg'] = str(int(work['yearend'])-1)
                if(work['monthbeg'] == '0'):
                    work['monthbeg'] = '1'
                if(work['monthend'] == '0'):
                    work['monthend'] = '12'
            #        print(work['monthbeg'] + " " + work['monthend'] + " " + work['yearbeg'] + " " + work['yearend'])
            #        print()
                start = datetime.datetime.strptime(work['monthbeg']+'.'+work['yearbeg'], "%m.%Y").date()
                finish = datetime.datetime.strptime(work['monthend']+'.'+work['yearend'], "%m.%Y").date()
                work['work_days'] = (finish-start+datetime.timedelta(days=1)).days
            #except:
             #   print((finish-start).days)
             #   return
f()

100%|██████████| 128945/128945 [00:19<00:00, 6551.78it/s]


In [42]:
pickle.dump(resume, open("resume_with_days.pkl", 'wb'))

In [55]:
a = 1
for res in resume:
    for work in res['work_history']:
        work['id'] = a
        a += 1

TypeError: 'float' object does not support item assignment

In [ ]:
for work in resume[0]['work_history']:
        #try:
            #print(work['monthbeg'] + " " + work['monthend'] + " " + work['yearbeg'] + " " + work['yearend'])
            if(work['monthbeg'] == '0'):
                work['monthbeg'] = '1'
            if(work['yearend'] == None):
                work['yearend'] = datetime.datetime.now().strftime("%Y")
                work['monthend'] = datetime.datetime.now().strftime("%m")
            if(work['monthend'] == '0'):
                work['monthend'] = '12'
            print(work['monthbeg'] + " " + work['monthend'] + " " + work['yearbeg'] + " " + work['yearend'])
            print()
            start = datetime.datetime.strptime(work['monthbeg']+work['yearbeg'], "%m%Y").date()
            finish = datetime.datetime.strptime(work['monthend']+work['yearend'], "%m%Y").date()
            work['work_months'] = str(int(str(finish-start+datetime.timedelta(days=1))[:str(finish-start).find(" ")])/30)
        #except:
        #    print(work)

In [23]:
resume[0]['']             

{'profession': 'Директор службы', 'name': '9c46c0cca3355f2ff25f59c7e7ea8191', 'company_scope': None, 'work': 'Управление подразделением компании и координация действий с другими подразделениями, маркетинг, поиск покупателей и проведение переговоров, составление и ведение контрактов, отбор товара, координация работ подразделений; открытие точек розничной торговли, управление торговой сетью, реорганизация склада, графики платежей и поставок, слежение за поставками, нормозапас, планирование, ценобразование, разработка и проведение рекламной компании, участие в выставках и ярмарках. ', 'town': 'Москва', 'type': 'Полная занятость', 'monthbeg': '9', 'yearbeg': '1999', 'monthend': '0', 'yearend': '0', 'is_last': '1'}


In [ ]:
resume[0][w]

In [70]:
a = 1
for res in tqdm(resume[0:1]):
    for work in res["work_history"]:
        print(res["work_history"][work]['id'])
        print(res["work_history"][work])


100%|██████████| 1/1 [00:00<00:00, 270.60it/s]

1
{'profession': 'Менеджер по клиентским отношениям', 'name': '336d370779b3261dcb00331fba5f7f47', 'company_scope': None, 'work': 'Ведение переговоров с потенциальными клиентами банка по поводу предоставления банковских услуг. ', 'town': 'Москва', 'type': 'Полная занятость', 'monthbeg': '2', 'yearbeg': '1997', 'monthend': '4', 'yearend': '1997', 'is_last': '0', 'work_months': datetime.timedelta(2), 'work_days': 60, 'id': 1}
2
{'profession': 'Экономист', 'name': '49748b69e3d3435cd98d241a6935f3d1', 'company_scope': None, 'work': 'Ведение складского учёта на ПК. ', 'town': 'Москва', 'type': 'Полная занятость', 'monthbeg': '6', 'yearbeg': '1997', 'monthend': '11', 'yearend': '1997', 'is_last': '0', 'work_months': datetime.timedelta(5, 11520), 'work_days': 154, 'id': 2}
3
{'profession': 'Экономист подразделения БиоХимИнвест', 'name': '7119f26423d0e892d1111d8a49372b15', 'company_scope': None, 'work': 'Документооборот, контроль выполнения договоров, мониторинг дилеров. ', 'town': 'Москва', 'ty

In [ ]:
list(firm_to_count.keys())[:10]

In [ ]:
jobs = get_jobs(0)

for i in range(len(vacancy)):
    #try:
        get_jobs(i)
    #except:
     #   print(i)
      #  break

#for job in jobs:
#    print(str(job) + " " + jobs[job]['profession'])
    #print(str(find_vac_number(jobs[job]['profession'])) + "  numb: " + str(firm_to_count[jobs[job]["name"]]))

In [60]:
resume[0]

{'additional_education': [],
 'age': 40,
 'base_education': [{'education_form': 'Дневная/Очная',
   'education_type': 'Высшее',
   'faculty': 'Экономический факультет',
   'institute': 'Московский государственный университет им. М.В. Ломоносова',
   'monthend': '0',
   'profession': 'Специализация "Экономика фирмы и отраслевых рынков" (магистр)',
   'town': 'Москва',
   'yearend': '2001'},
  {'education_form': 'Дневная/Очная',
   'education_type': 'Высшее',
   'faculty': 'Экономический факультет',
   'institute': 'Московский государственный университет им. М.В. Ломоносова',
   'monthend': '0',
   'profession': 'Общая экономика (бакалавр)',
   'town': 'Москва',
   'yearend': '1999'}],
 'best': 'Компьютерные навыки:\nMS Excel 2007 (продвинутый пользователь, в т.ч. сводные таблицы), MS Word 2007 (продвинутый пользователь), MS PowerPoint 2007 (уверенный пользователь), MS Access 2007, 1С-Предприятие 7.7, 8.1 (уверенный пользователь), Справочная правовая система КонсультантПлюс 3000, Электро

In [ ]:
get_jobs(1)